# Variables a explicar: USDT y BTC

In [9]:
pip install boto3 

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import boto3
import os
from io import BytesIO
from zipfile import ZipFile

In [11]:
# Crear una instancia de sesión de AWS utilizando el perfil especificado en el archivo credentials
session = boto3.Session(profile_name='aws')

# Crear una instancia de cliente de S3 utilizando la sesión creada
s3 = session.client('s3')

# Definir el nombre del archivo ZIP en S3 y el nombre del archivo CSV dentro del ZIP
nombre_archivo_zip = "bcra-data-delivery-2024-04-25.zip"
nombre_archivo_csv = "bcra-data-delivery-2024-04-25/raw_market_candles_1d.csv"

# Descargar el archivo ZIP desde S3
objeto_zip = s3.get_object(Bucket='coinmetrics.io', Key=nombre_archivo_zip)

# Leer el contenido del archivo ZIP en memoria
contenido_zip = BytesIO(objeto_zip['Body'].read())

# Leer el archivo CSV desde el archivo ZIP en memoria
with ZipFile(contenido_zip) as archivo_zip:
    with archivo_zip.open(nombre_archivo_csv) as csv_file:
        # Cargar el archivo CSV en un DataFrame de pandas
        df = pd.read_csv(csv_file)

# Dividir la columna 'market' en 'exchange', 'crypto', 'fiat' y 'market_type'
df[['exchange', 'crypto', 'fiat', 'market_type']] = df['market'].str.split('-', expand=True)

# Convertir la columna 'time' a tipo datetime si no lo está
df['time'] = pd.to_datetime(df['time'])

# Agrupar por exchange, crypto, moneda fiat, tipo de mercado y mes, y sumar el volumen en dólares
df_agrupado = df.groupby([df['exchange'], df['crypto'], df['fiat'], df['market_type'], df['time'].dt.to_period('M')])['candle_usd_volume'].sum()

/tmp/ipykernel_2126/219889092.py:30: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [12]:
df_agrupado_reset = df_agrupado.reset_index()

In [13]:
import plotly.express as px
import pandas as pd

df = df_agrupado_reset

# Crear el Treemap
fig = px.treemap(df, path=['fiat', 'crypto', 'exchange'], values='candle_usd_volume')

# Mostrar el Treemap
fig.show()


In [14]:
# Filtrar los datos para mantener solo las combinaciones de monedas fiat con las criptomonedas USDT y BTC
df_agrupado_usdt_btc = df_agrupado[df_agrupado.index.get_level_values('crypto').isin(['usdt', 'btc'])]

In [15]:
# Convertir la Serie df_agrupado en un DataFrame
df_agrupado_usdt_btc = df_agrupado_usdt_btc.reset_index()

In [16]:
df_agrupado_usdt_btc_cft = df_agrupado_usdt_btc.groupby(['crypto', 'fiat', 'time']).sum()
# Convertir la Serie df_agrupado en un DataFrame
df_agrupado_usdt_btc_cft = df_agrupado_usdt_btc_cft.reset_index()

In [17]:
# Paso 1: Realizar una operación de pivote en el DataFrame
df_agrupado_usdt_btc_cft_pivot = df_agrupado_usdt_btc_cft.pivot_table(index=['fiat', 'time'], columns='crypto', values='candle_usd_volume')

In [18]:
df_agrupado_usdt_btc_cft_pivot = df_agrupado_usdt_btc_cft_pivot.reset_index()

In [19]:
# Calcular la nueva columna de proporción de USDT sobre la suma total
df_agrupado_usdt_btc_cft_pivot['usdt_ratio'] = df_agrupado_usdt_btc_cft_pivot['usdt'] / (df_agrupado_usdt_btc_cft_pivot['usdt'] + df_agrupado_usdt_btc_cft_pivot['btc'])

In [20]:
# Eliminar las filas que contienen valores NaN en la columna 'usdt_ratio'
df_agrupado_usdt_btc_cft_pivot.dropna(subset=['usdt_ratio'], inplace=True)

In [21]:
import plotly.graph_objects as go

# Convertir las fechas a formato string
df_agrupado_usdt_btc_cft_pivot['time'] = df_agrupado_usdt_btc_cft_pivot['time'].dt.strftime('%Y-%m')

# Filtrar el DataFrame para eliminar NaNs en la columna usdt_ratio
df_filtered = df_agrupado_usdt_btc_cft_pivot.dropna(subset=['usdt_ratio'])

# Crear el gráfico de líneas
fig = go.Figure()

# Iterar sobre cada fiat
for fiat in df_filtered['fiat'].unique():
    # Filtrar el DataFrame por fiat
    df_fiat = df_filtered[df_filtered['fiat'] == fiat]
    
    # Añadir una línea al gráfico para el fiat actual
    fig.add_trace(go.Scatter(x=df_fiat['time'], y=df_fiat['usdt_ratio'], mode='lines', name=fiat))

# Personalizar el diseño del gráfico
fig.update_layout(title='Evolución del volumen relativo de USDT por fiat en el tiempo',
                   xaxis_title='Fecha',
                   yaxis_title='Volumen relativo de USDT',
                   legend_title='Fiat')

# Mostrar el gráfico
fig.show()

In [22]:
import plotly.graph_objects as go

# Filtrar los datos para mantener solo las combinaciones de monedas fiat con la criptomoneda USDT
df_agrupado_usdt = df_agrupado[df_agrupado.index.get_level_values('crypto') == 'usdt']

# Convertir la Serie df_agrupado en un DataFrame
df_agrupado_usdt = df_agrupado_usdt.reset_index()

# Crear el gráfico de barras apiladas con Plotly
fig = go.Figure()

for fiat in df_agrupado_usdt['fiat'].unique():
    data = df_agrupado_usdt[df_agrupado_usdt['fiat'] == fiat]
    fig.add_trace(go.Bar(name=f"{fiat}", x=data['time'].dt.strftime('%Y-%m'), y=data['candle_usd_volume']))

fig.update_layout(
    title='Evolución Mensual de Volumen en USD de USDT por Moneda Fiat',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Volumen en USD'),
    barmode='stack',
)
fig.show()

# Variables explicativas

https://www.bd-econ.com/imfapi1.html

# Para buscar el nombre de las series

https://data.imf.org/regular.aspx?key=34426

e = ENDE_XDC_USD_RATE

Alternativa: 

https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sid=1390030341854
buscar la serie, botón derecho, code



In [23]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET  # Importar el módulo necesario para parsear XML
import time  # Importar el módulo para manejar intervalos de tiempo

def obtener_datos_economicos(url_base, variables, paises, start_period, end_period):
    """
    Obtiene datos económicos para una lista de países y una lista de variables específicas en un rango de tiempo específico.
    
    Args:
        url_base (str): URL base para las consultas.
        variables (list): Lista de códigos de variables a consultar.
        paises (list): Lista de códigos de país.
        start_period (str): Periodo de inicio en formato 'YYYY' para variables mensuales o 'YYYY-Q' para variables trimestrales.
        end_period (str): Periodo de fin en formato 'YYYY' para variables mensuales o 'YYYY-Q' para variables trimestrales.
    
    Returns:
        pandas.DataFrame: Un DataFrame con los datos económicos para cada país, variable y período de tiempo.
    """
    datos_economicos = []

    for variable in variables:
        for pais in paises:
            # Construir la URL de la consulta para la variable y país actual
            url = f"{url_base}/M.{pais}.{variable}?startPeriod={start_period}&endPeriod={end_period}&dimensionAtObservation=M"
            
            # Realizar la solicitud HTTP y obtener los datos
            response = requests.get(url)
            
            # Verificar si la solicitud fue exitosa
            if response.status_code == 200:
                # Analizar el XML y obtener los datos económicos
                root = ET.fromstring(response.content)
                for obs in root.findall('.//{http://dataservices.imf.org/compact/IFS}Obs'):
                    time_period = obs.attrib['TIME_PERIOD']
                    obs_value = float(obs.attrib['OBS_VALUE'])
                    datos_economicos.append((pais, time_period, variable, obs_value))
            else:
                print(f"Error al obtener datos para {variable} - {pais}: {response.status_code}")
            
            # Esperar 1 segundo antes de la siguiente solicitud para evitar sobrecargar el servidor
            time.sleep(2)
    
    # Crear un DataFrame a partir de la lista de datos económicos
    df = pd.DataFrame(datos_economicos, columns=['País', 'Fecha', 'Variable', 'Valor'])
    
    return df

# Ejemplo de uso
url_base = "http://dataservices.imf.org/REST/SDMX_XML.svc/CompactData/IFS"
variables = ["PCPI_IX", "EREER_IX"]
paises = ["AU", "US", "JP", "GB", "CA", "AR", "BR", "NG", "PL", "RO", "RU", "ZA", "TR", "UA"]#, "IN"]
start_period = "2020"
end_period = "2024"
datos_economicos_panel = obtener_datos_economicos(url_base, variables, paises, start_period, end_period)


# Base de datos combinada

In [ ]:
# Seleccionar solo las columnas relevantes para el volumen mensual de USDT por país
df_usdt_simplificado = df_agrupado_usdt[['fiat', 'time', 'candle_usd_volume']]

In [ ]:
# Mapeo de moneda a país
mapeo_moneda_pais = {
    'aud': 'AU',
    'usd': 'US',
    'jpy': 'JP',
    'gbp': 'GB',
    'eur': 'EU',
    'cad': 'CA',
    'ars': 'AR',
    'brl': 'BR',
    'ngn': 'NG',
    'pln': 'PL',
    'ron': 'RO',
    'rub': 'RU',
    'zar': 'ZA',
    'try': 'TR',
    'uah': 'UA',
    'inr': 'IN'
}

# Reemplazar las monedas por los nombres de los países en la columna 'fiat'
df_usdt_simplificado['fiat'] = df_usdt_simplificado['fiat'].replace(mapeo_moneda_pais)

C:\Users\Emiliano Giupponi\AppData\Local\Temp\ipykernel_10096\1518727134.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df

# Crear un diccionario con los nuevos nombres de las columnas
nuevos_nombres = {
    'fiat': 'País',
    'time': 'Fecha',
    'candle_usd_volume': 'Valor',
}

# Renombrar las columnas
df_usdt_simplificado = df_usdt_simplificado.rename(columns=nuevos_nombres)

# Ahora tu DataFrame tendrá las columnas con los nuevos nombres

In [ ]:
import pandas as pd

# Suponiendo que tienes los DataFrames df_usdt_simplificado y datos_economicos_panel

# Convertir la columna 'Fecha' a tipo datetime en ambos DataFrames
df_usdt_simplificado['Fecha'] = pd.to_datetime(df_usdt_simplificado['Fecha'].astype(str))
datos_economicos_panel['Fecha'] = pd.to_datetime(datos_economicos_panel['Fecha'].astype(str))

# Unir los DataFrames por las columnas 'País' y 'Fecha'
df_merged = pd.merge(df_usdt_simplificado, datos_economicos_panel, on=['País', 'Fecha'], how='inner')

# Ahora tu DataFrame df_merged contendrá los datos combinados con las fechas alineadas


In [ ]:
df_pivot = df_merged.pivot_table(index=['País', 'Fecha', 'Valor_x'], columns='Variable', values='Valor_y').reset_index()

In [ ]:
def clasificar_economia_por_codigo(codigo_pais):
    # Diccionario que asigna códigos de país a clasificaciones económicas
    clasificacion_por_codigo = {
        'AU': 'AE',  # Australia - Economía avanzada (AE)
        'US': 'AE',  # Estados Unidos - Economía avanzada (AE)
        'JP': 'AE',  # Japón - Economía avanzada (AE)
        'GB': 'AE',  # Reino Unido - Economía avanzada (AE)
        'CA': 'AE',  # Canadá - Economía avanzada (AE)
        'AR': 'EMDE',  # Argentina - Economía de mercado emergente y en desarrollo (EMDE)
        'BR': 'EMDE',  # Brasil - Economía de mercado emergente y en desarrollo (EMDE)
        'NG': 'EMDE',  # Nigeria - Economía de mercado emergente y en desarrollo (EMDE)
        'PL': 'EMDE',  # Polonia - Economía de mercado emergente y en desarrollo (EMDE)
        'RO': 'EMDE',  # Rumania - Economía de mercado emergente y en desarrollo (EMDE)
        'RU': 'EMDE',  # Rusia - Economía de mercado emergente y en desarrollo (EMDE)
        'ZA': 'EMDE',  # Sudáfrica - Economía de mercado emergente y en desarrollo (EMDE)
        'TR': 'EMDE',  # Turquía - Economía de mercado emergente y en desarrollo (EMDE)
        'UA': 'EMDE',  # Ucrania - Economía de mercado emergente y en desarrollo (EMDE)
        'IN': 'EMDE'   # India - Economía de mercado emergente y en desarrollo (EMDE)
    }

    # Obtener la clasificación económica correspondiente al código de país
    return clasificacion_por_codigo.get(codigo_pais, 'No definido')

# Aplicar la función a la columna 'País' para obtener la clasificación
df_pivot['Clasificación_económica'] = df_pivot['País'].apply(clasificar_economia_por_codigo)


In [ ]:
# Filtrar los datos por la clasificación económica
emde_data = df_pivot[df_pivot['Clasificación_económica'] == 'EMDE']
ad_data = df_pivot[df_pivot['Clasificación_económica'] == 'AE']

# Calcular estadísticas descriptivas para Valor_x
emde_stats = emde_data['Valor_x'].describe()
ad_stats = ad_data['Valor_x'].describe()

print("Estadísticas descriptivas para países emergentes (EMDE):\n", emde_stats)
print("\nEstadísticas descriptivas para países avanzados (Ad):\n", ad_stats)


Estadísticas descriptivas para países emergentes (EMDE):
 count    3.640000e+02
mean     3.034121e+08
std      8.036116e+08
min      0.000000e+00
25%      1.163026e+06
50%      1.033001e+07
75%      1.115008e+08
max      5.604019e+09
Name: Valor_x, dtype: float64

Estadísticas descriptivas para países avanzados (Ad):
 count    7.380000e+02
mean     6.098780e+08
std      1.361231e+09
min      6.931899e+03
25%      8.731509e+06
50%      5.157219e+07
75%      2.968823e+08
max      1.262859e+10
Name: Valor_x, dtype: float64


In [ ]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import statsmodels.api as sm

# Eliminamos las filas con valores NaN, ya que la regresión lineal no puede manejarlos
df_regression = df_pivot.dropna(subset=['Valor_x', 'PCPI_IX', 'Clasificación_económica'])

# Definimos las variables independientes (EREER_IX y PCPI_IX)
X = df_regression['PCPI_IX', 'Clasificación_económica']

# Añadimos una constante a la matriz X para la intercepción
X = sm.add_constant(X)

# Definimos la variable dependiente (Valor_x)
y = df_regression['Valor_x']

# Creamos el modelo de regresión lineal
model = sm.OLS(y, X)

# Ajustamos el modelo a los datos
results = model.fit()

# Imprimimos los resultados de la regresión
print(results.summary())


KeyError: ('PCPI_IX', 'Clasificación_económica')